In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
from sklearn.impute import SimpleImputer, KNNImputer


In [ ]:

# Veri setini yükle
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
sutunlar = [
    "yas", "calisma_durumu", "agirlik_faktoru", "egitim", "egitim_sure", "medeni_durum",
    "meslek", "akrabalik_durumu", "irk", "cinsiyet", "sermaye_getirisi", "sermaye_kaybi",
    "haftalik_calisma_saati", "ulke", "gelir"
]
df = pd.read_csv(url, names=sutunlar, na_values=" ?", skipinitialspace=True)
df.head()


In [ ]:

# Eksik veri kontrolü
print("Eksik veriler:")
print(df.isnull().sum())


In [ ]:

# Sayısal ve kategorik değişkenleri belirle
sayisal_degiskenler = ["yas", "agirlik_faktoru", "egitim_sure", "sermaye_getirisi", "sermaye_kaybi", "haftalik_calisma_saati"]
kategorik_degiskenler = ["calisma_durumu", "egitim", "medeni_durum", "meslek", "akrabalik_durumu", "irk", "ulke", "cinsiyet", "gelir"]


In [ ]:

# Eksik verileri doldurma (Sadece sayısal değişkenlerde uyguluyoruz)
imputer_median = SimpleImputer(strategy='median')
df[sayisal_degiskenler] = imputer_median.fit_transform(df[sayisal_degiskenler])

imputer_knn = KNNImputer(n_neighbors=3)
df[sayisal_degiskenler] = imputer_knn.fit_transform(df[sayisal_degiskenler])


In [ ]:

# Kategorik değişkenleri sayısal hale dönüştürme
le = LabelEncoder()
df["cinsiyet"] = le.fit_transform(df["cinsiyet"])
df["gelir"] = df["gelir"].map({"<=50K": 0, ">50K": 1})


In [ ]:

# One-Hot Encoding ile dönüşüm (İlk kategori düşürülerek dummy değişken tuzağı önlenir)
df = pd.get_dummies(df, columns=["calisma_durumu", "egitim", "medeni_durum", "meslek", "akrabalik_durumu", "irk", "ulke"], drop_first=True)


In [ ]:

# Sayısal değişkenleri ölçekleme (Sadece StandardScaler kullanıyoruz)
scaler = StandardScaler()
df[sayisal_degiskenler] = scaler.fit_transform(df[sayisal_degiskenler])


In [ ]:

# Eğitim ve test veri setine ayırma
X = df.drop("gelir", axis=1)
y = df["gelir"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:

# Lojistik regresyon modeli eğitme
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)


In [ ]:

# Modelin performansını değerlendirme
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Doğruluk (Accuracy):", accuracy)
print("F1 Skoru:", f1)
